# QKDpy Tutorial: Quantum Key Distribution Simulation

Welcome to this interactive tutorial on QKDpy, a Python library for Quantum Key Distribution simulations. In this tutorial, you'll learn how to use QKDpy to simulate various QKD protocols and analyze their performance.

## Table of Contents
1. [Introduction to QKD](#introduction)
2. [Installation](#installation)
3. [Basic QKD Simulation](#basic-simulation)
4. [Advanced Protocols](#advanced-protocols)
5. [Quantum Networks](#quantum-networks)
6. [Visualization Tools](#visualization)
7. [Machine Learning Integration](#ml-integration)

<a id='introduction'></a>
## 1. Introduction to QKD

Quantum Key Distribution (QKD) is a method of secure communication that uses quantum mechanics to enable two parties to produce a shared random secret key known only to them, which can then be used to encrypt and decrypt messages. QKDpy provides implementations of various QKD protocols, including:

- BB84: The first and most well-known QKD protocol
- E91: Based on quantum entanglement
- SARG04: A protocol resistant to photon number splitting attacks
- CV-QKD: Continuous-variable QKD
- HD-QKD: High-dimensional QKD

Let's start by importing the necessary modules:

In [ ]:
# Import QKDpy modules
import numpy as np
import matplotlib.pyplot as plt

# Core QKDpy modules
from qkdpy import BB84, E91, QuantumChannel, Qubit
from qkdpy.utils import ProtocolVisualizer, KeyRateAnalyzer
from qkdpy.network import QuantumNetwork
from qkdpy.ml import QKDOptimizer, QKDAnomalyDetector

# For better visualization
plt.style.use('seaborn-v0_8')
%matplotlib inline

<a id='installation'></a>
## 2. Installation

To install QKDpy, you can use pip:

```bash
pip install qkdpy
```

Or if you prefer using uv (recommended):

```bash
uv pip install qkdpy
```

For development installation:

```bash
git clone https://github.com/Pranava-Kumar/qkdpy.git
cd qkdpy
uv pip install -e .
```

<a id='basic-simulation'></a>
## 3. Basic QKD Simulation

Let's start with a basic simulation of the BB84 protocol. We'll create a quantum channel, instantiate the BB84 protocol, and execute it to generate a shared key.

In [ ]:
# Create a quantum channel with some noise and loss
channel = QuantumChannel(loss=0.1, noise_model='depolarizing', noise_level=0.05)

# Create a BB84 protocol instance
bb84 = BB84(channel, key_length=128)

# Execute the protocol
results = bb84.execute()

# Print the results
print(f"Generated key length: {len(results['final_key'])}")
print(f"QBER: {results['qber']:.4f}")
print(f"Is secure: {results['is_secure']}")
print(f"Final key (first 20 bits): {results['final_key'][:20]}")

### Visualizing the Protocol Execution

QKDpy provides powerful visualization tools to help understand how the protocol works:

In [ ]:
# Visualize the BB84 protocol execution
fig = ProtocolVisualizer.plot_bb84_protocol(
    alice_bits=bb84.alice_bits[:50],
    alice_bases=bb84.alice_bases[:50],
    bob_bases=bb84.bob_bases[:50],
    bob_results=bb84.bob_results[:50],
    title="BB84 Protocol Execution Visualization"
)
plt.show()

<a id='advanced-protocols'></a>
## 4. Advanced Protocols

QKDpy implements several advanced QKD protocols. Let's explore some of them:

In [ ]:
# High-Dimensional QKD (HD-QKD)
from qkdpy.protocols.hd_qkd import HDQKD

# Create a quantum channel
channel = QuantumChannel(loss=0.1, noise_model='depolarizing', noise_level=0.05)

# Create an HD-QKD protocol instance with 4-dimensional qudits
hd_qkd = HDQKD(channel, key_length=100, dimension=4)

# Execute the protocol
results = hd_qkd.execute()

# Print the results
print(f"Generated key: {results['final_key'][:20]}...")
print(f"QBER: {results['qber']:.4f}")
print(f"Is secure: {results['is_secure']}")
print(f"Dimensional efficiency gain: {hd_qkd.get_dimension_efficiency():.2f}x")

### Comparing Protocol Performance

Let's compare the performance of different protocols:

In [ ]:
# Compare different protocols
protocols = [
    ("BB84", BB84(channel, key_length=100)),
    ("E91", E91(channel, key_length=100)),
    ("HD-QKD", HDQKD(channel, key_length=100, dimension=4))
]

# Execute each protocol and collect results
results = {}
for name, protocol in protocols:
    try:
        result = protocol.execute()
        results[name] = {
            'key_length': len(result['final_key']),
            'qber': result['qber'],
            'is_secure': result['is_secure'],
            'execution_time': result.get('execution_time', 0)
        }
    except Exception as e:
        print(f"Error executing {name}: {e}")
        results[name] = None

# Display results
print("Protocol Comparison Results:")
print("-" * 50)
for name, result in results.items():
    if result:
        print(f"{name}: Key length={result['key_length']}, QBER={result['qber']:.4f}, Secure={result['is_secure']}")
    else:
        print(f"{name}: Failed to execute")

<a id='quantum-networks'></a>
## 5. Quantum Networks

QKDpy also provides tools for simulating quantum networks with multiple nodes:

In [ ]:
# Create a quantum network
network = QuantumNetwork("Tutorial Network")

# Add nodes with different protocols
channel1 = QuantumChannel(loss=0.1, noise_model='depolarizing', noise_level=0.05)
channel2 = QuantumChannel(loss=0.15, noise_model='depolarizing', noise_level=0.07)
channel3 = QuantumChannel(loss=0.05, noise_model='depolarizing', noise_level=0.03)

# Create protocols for each node
bb84_alice = BB84(channel1, key_length=128)
bb84_bob = BB84(channel1, key_length=128)
bb84_charlie = BB84(channel2, key_length=128)

# Add nodes to the network
network.add_node("Alice", bb84_alice)
network.add_node("Bob", bb84_bob)
network.add_node("Charlie", bb84_charlie)

# Add connections
network.add_connection("Alice", "Bob", channel1)
network.add_connection("Bob", "Charlie", channel2)
network.add_connection("Alice", "Charlie", channel3)

# Display network information
stats = network.get_network_statistics()
print(f"Network: {stats['network_name']}")
print(f"Number of nodes: {stats['num_nodes']}")
print(f"Number of connections: {stats['num_connections']}")
print(f"Average degree: {stats['average_degree']:.2f}")

### Establishing Keys in the Network

Let's establish keys between different nodes in the network:

In [ ]:
# Establish a key between Alice and Charlie
key = network.establish_key_between_nodes("Alice", "Charlie", key_length=64)

if key:
    print(f"Successfully established key between Alice and Charlie:")
    print(f"Key (first 20 bits): {key[:20]}")
else:
    print("Failed to establish key between Alice and Charlie")

# Simulate network performance
performance = network.simulate_network_performance(num_trials=50)
print(f"\nNetwork Performance Simulation:")
print(f"Success rate: {performance['success_rate']:.2%}")
print(f"Average key length: {performance['average_key_length']:.1f}")
print(f"Average QBER: {performance['average_qber']:.4f}")

<a id='visualization'></a>
## 6. Visualization Tools

QKDpy includes advanced visualization tools to help understand quantum states and protocol execution:

In [ ]:
# Visualize quantum states on the Bloch sphere
from qkdpy.utils import BlochSphere

# Create different quantum states
qubit_zero = Qubit.zero()
qubit_one = Qubit.one()
qubit_plus = Qubit.plus()
qubit_minus = Qubit.minus()

# Plot multiple qubit states on the Bloch sphere
qubits = [qubit_zero, qubit_one, qubit_plus, qubit_minus]
labels = ["|0⟩", "|1⟩", "|+⟩", "|-⟩"]

fig = BlochSphere.plot_multiple_qubits(qubits, labels, title="Quantum States on Bloch Sphere")
plt.show()

### Key Rate Analysis

Analyze how key generation rate varies with different parameters:

In [ ]:
# Analyze key rate vs. loss
from qkdpy.utils import KeyRateAnalyzer

# Test different loss values
loss_values = np.linspace(0, 0.5, 20)
key_rates = []

for loss in loss_values:
    channel = QuantumChannel(loss=loss, noise_model='depolarizing', noise_level=0.05)
    bb84 = BB84(channel, key_length=100)
    try:
        results = bb84.execute()
        key_rates.append(bb84.get_key_rate())
    except:
        key_rates.append(0)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(loss_values, key_rates, 'b-o', linewidth=2, markersize=4)
plt.xlabel('Channel Loss')
plt.ylabel('Key Rate')
plt.title('Key Rate vs. Channel Loss')
plt.grid(True, alpha=0.3)
plt.show()

<a id='ml-integration'></a>
## 7. Machine Learning Integration

QKDpy integrates machine learning tools for protocol optimization and anomaly detection:

In [ ]:
# Optimize QKD protocol parameters using machine learning
from qkdpy.ml import QKDOptimizer

# Define an objective function to maximize
def objective_function(params):
    # Extract parameters
    loss = params.get('loss', 0.1)
    noise_level = params.get('noise_level', 0.05)
    
    # Create channel and protocol
    channel = QuantumChannel(loss=loss, noise_model='depolarizing', noise_level=noise_level)
    protocol = BB84(channel, key_length=100)
    
    # Execute protocol and return key rate (higher is better)
    try:
        results = protocol.execute()
        return protocol.get_key_rate() if results.get('is_secure', False) else 0
    except:
        return 0

# Define parameter space
parameter_space = {
    'loss': (0.0, 0.5),
    'noise_level': (0.0, 0.2)
}

# Create optimizer
optimizer = QKDOptimizer("BB84")

# Optimize parameters
results = optimizer.optimize_channel_parameters(
    parameter_space=parameter_space,
    objective_function=objective_function,
    num_iterations=20,
    method="bayesian"
)

print("Optimization Results:")
print(f"Best parameters: {results['best_parameters']}")
print(f"Best objective value: {results['best_objective_value']:.4f}")

### Anomaly Detection

Detect anomalies in QKD system performance:

In [ ]:
# Anomaly detection for QKD systems
from qkdpy.ml import QKDAnomalyDetector

# Create historical data
history = [
    {'qber': 0.02, 'key_rate': 1000, 'loss': 0.1},
    {'qber': 0.03, 'key_rate': 950, 'loss': 0.12},
    {'qber': 0.025, 'key_rate': 980, 'loss': 0.11},
    {'qber': 0.028, 'key_rate': 960, 'loss': 0.115},
    {'qber': 0.022, 'key_rate': 1020, 'loss': 0.09},
]

# Establish baseline
detector = QKDAnomalyDetector()
detector.establish_baseline(history)

# Test with normal metrics
normal_metrics = {'qber': 0.025, 'key_rate': 970, 'loss': 0.11}
anomalies = detector.detect_anomalies(normal_metrics)
print(f"Normal metrics anomalies: {anomalies}")

# Test with anomalous metrics
anomalous_metrics = {'qber': 0.5, 'key_rate': 100, 'loss': 0.8}
anomalies = detector.detect_anomalies(anomalous_metrics)
print(f"Anomalous metrics anomalies: {anomalies}")

# Get detection report
report = detector.get_detection_report()
print(f"Detection report: {report}")

## Conclusion

In this tutorial, we've explored the key features of QKDpy:

1. **Protocol Implementation**: We demonstrated how to use various QKD protocols including BB84, E91, and HD-QKD.
2. **Quantum Networks**: We showed how to create and simulate quantum networks with multiple nodes.
3. **Visualization Tools**: We used advanced visualization tools to understand quantum states and protocol execution.
4. **Machine Learning Integration**: We demonstrated how to optimize protocols and detect anomalies using ML tools.

QKDpy provides a comprehensive platform for quantum cryptography research and education. Whether you're a researcher exploring new protocols or a student learning about quantum cryptography, QKDpy offers the tools you need to simulate and analyze QKD systems.

For more information, please visit the [QKDpy GitHub repository](https://github.com/Pranava-Kumar/qkdpy) or check out the documentation.